
# Performance And Costing Analysis of Ergo Node

## Introduction
Thds notebook uses `Metrics.sqlite` database collected during full node syncronization.
See `README.md` for detail of database schema.



## Setup
If necessary uncomment and run the following commands to setup necessary packages.


In [15]:
#!pip install ipython-sql
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
!jupyter nbextension enable python-markdown/main


Enabling notebook extension python-markdown/main...
      - Validating: OK


In [16]:
# open sqlite connection to perform helper queries
import pandas as pd
import sqlite3
conn = sqlite3.connect("../Metrics.sqlite")

In [17]:
checks = dict(
    appendFullBlockOk=pd.read_sql_query(f"""
        -- validate appendFullBlock tables
        select *
        from appendFullBlock a1
                 join appendFullBlock4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    applyTransactionsOk = pd.read_sql_query(f"""
        select *
        from applyTransactions a1
                 join applyTransactions4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0,
    createUtxoStateOk = pd.read_sql_query(f"""
        select *
        from createUtxoState a1
                 join createUtxoState4 a2 on a1.blockId = a2.blockId
        where a1.height != a2.height
           or a1.tx_num != a2.tx_num;
        """, conn).size == 0
)
if not (checks.get("appendFullBlockOk") and checks.get("applyTransactionsOk") and checks.get("createUtxoStateOk")):
  ok = checks
else:
  ok = "ok"

## Sanity checks: {{ ok }}
